In [ ]:
import sys
from env_key_manager import APIKeyManager

# 创建实例
key_manager = APIKeyManager()

# 设置环境变量
key_manager.setup_api_key(["DEEPSEEK_API_KEY", "LANGSMITH_TRACING", "LANGSMITH_PROJECT", ])

# 查看Python版本
!python -V
# 查看安装的库
if 'win' in sys.platform.lower():
    !pip list | findstr "lang openai llm tiktoken chromadb cryptography duck unstructured numpy scipy"
else:
    !pip list | grep -E 'lang|openai|llm|tiktoken|chromadb|cryptography|duck|unstructured|numpy|scipy'

Python 3.10.16
cryptography                      44.0.2
duckduckgo_search                 6.3.7
langchain                         0.3.19
langchain-community               0.3.18
langchain-core                    0.3.49
langchain-deepseek                0.1.3
langchain-openai                  0.3.11
langchain-text-splitters          0.3.6
langgraph                         0.3.21
langgraph-checkpoint              2.0.23
langgraph-prebuilt                0.1.7
langgraph-sdk                     0.1.60
langserve                         0.3.1
langsmith                         0.3.8
numpy                             1.26.4
openai                            1.69.0
scipy                             1.15.2
tiktoken                          0.9.0


# 使用聊天模型和提示模板构建一个简单的LLM应用

在本快速入门指南中，我们将向您展示如何使用LangChain构建一个简单的LLM应用程序。该应用程序能够将文本从英语翻译成其他语言。这是一个相对简单的LLM应用——仅需一次LLM调用加上适当的提示即可完成。尽管如此，这仍是入门LangChain的绝佳方式——仅通过提示设计和LLM调用就能实现许多功能！

阅读本教程后，您将对以下内容有高层次的理解：

- 使用[语言模型](/docs/concepts/chat_models)

- 使用[提示模板](/docs/concepts/prompt_templates)

- 使用 [LangSmith](https://docs.smith.langchain.com/) 调试和追踪您的应用程序

让我们开始吧！

## 安装设置

### Jupyter Notebook

本教程及其他教程或许在 [Jupyter notebooks](https://jupyter.org/) 中运行最为便捷。在交互式环境中学习教程是深入理解内容的绝佳方式。安装方法请参阅 [此处](https://jupyter.org/install)。

### 安装

要安装LangChain，请运行：

<!-- HIDE_IN_NB
import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
<CodeBlock language="bash">pip install langchain</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
<CodeBlock language="bash">conda install langchain -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>
HIDE_IN_NB -->

In [1]:
# | output: false

# %pip install langchain
# OR
# %conda install langchain -c conda-forge

更多详情，请参阅我们的[安装指南](/docs/how_to/installation)。

### LangSmith

使用LangChain构建的许多应用程序将包含多个步骤，涉及多次LLM调用。
随着这些应用变得越来越复杂，能够检查链或代理内部的具体运行情况变得至关重要。
最佳方式是使用 [LangSmith](https://smith.langchain.com)。

在以上链接完成注册后，请确保设置环境变量以开始记录追踪数据：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
export LANGSMITH_PROJECT="default" # 或其他任意项目名称
```

In [4]:
# import getpass
# import os

# try:
#     # load environment variables from .env file (requires `python-dotenv`)
#     from dotenv import load_dotenv

#     load_dotenv()
# except ImportError:
#     pass

# os.environ["LANGSMITH_TRACING"] = "true"
# if "LANGSMITH_API_KEY" not in os.environ:
#     os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
#         prompt="Enter your LangSmith API key (optional): "
#     )
# if "LANGSMITH_PROJECT" not in os.environ:
#     os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
#         prompt='Enter your LangSmith Project Name (default = "default"): '
#     )
#     if not os.environ.get("LANGSMITH_PROJECT"):
#         os.environ["LANGSMITH_PROJECT"] = "default"
# if "OPENAI_API_KEY" not in os.environ:
#     os.environ["OPENAI_API_KEY"] = getpass.getpass(
#         prompt="Enter your OpenAI API key (required if using OpenAI): "
#     )

## 使用语言模型

首先，我们来学习如何单独使用语言模型。LangChain支持多种可互换使用的语言模型。有关特定模型的入门指南，请参阅[支持的集成](/docs/integrations/chat/)。

<!-- HIDE_IN_NB>
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs overrideParams={{openai: {model: "gpt-4o-mini"}}} />
HIDE_IN_NB -->

In [3]:
# | output: false
# | echo: false

from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(model="deepseek-chat")

让我们首先直接使用这个模型。[聊天模型](/docs/concepts/chat_models)是LangChain中[可运行对象](/docs/concepts/runnables/)的实例，这意味着它们提供了与之交互的标准接口。要简单地调用模型，我们可以向`.invoke`方法传入一个[消息](/docs/concepts/messages/)列表。

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao! 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 12, 'total_tokens': 17, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 12}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3d5141a69a_prod0225', 'id': '369d6d54-9756-4d16-b783-79f0798789db', 'finish_reason': 'stop', 'logprobs': None}, id='run-058a3cd1-887f-4e47-938e-f3eba48d7499-0', usage_metadata={'input_tokens': 12, 'output_tokens': 5, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

:::提示

如果已启用LangSmith，我们可以看到此次运行已记录到LangSmith中，并能查看[LangSmith追踪记录](https://smith.langchain.com/public/88baa0b2-7c1a-4d09-ba30-a47985dde2ea/r)。该追踪记录会报告[令牌](/docs/concepts/tokens/)使用情况、延迟时间、[标准模型参数](/docs/concepts/chat_models/#standard-parameters)（如temperature）以及其他信息。

:::

请注意，ChatModels 接收[消息](/docs/concepts/messages/)对象作为输入，并生成消息对象作为输出。除了文本内容外，消息对象还传递对话[角色](/docs/concepts/messages/#role)，并包含重要数据，例如[工具调用](/docs/concepts/tool_calling/)和令牌使用计数。

LangChain 还支持通过字符串或 [OpenAI 格式](/docs/concepts/messages/#openai-format) 输入聊天模型。以下是等效的：

```python
model.invoke("你好")

model.invoke([{"role": "user", "content": "你好"}])

model.invoke([HumanMessage("你好")])
```
### 流式传输

由于聊天模型属于[可运行对象](/docs/concepts/runnables/)，它们提供了一套标准接口，包含异步和流式调用模式。这使得我们可以从聊天模型中逐令牌流式获取输出：

In [5]:
for token in model.stream(messages):
    print(token.content, end="|")

|C|iao|!| 😊||

您可以在[本指南](/docs/how_to/chat_streaming/)中找到关于流式聊天模型输出的更多详细信息。

## 提示模板

目前我们直接将一组消息传递给语言模型。这组消息从何而来？通常，它是由用户输入和应用程序逻辑共同构建而成。应用程序逻辑通常会接收原始用户输入，并将其转换为一组可直接传递给语言模型的消息。常见的转换操作包括添加系统消息，或根据用户输入格式化模板。

[提示模板](/docs/concepts/prompt_templates/)是LangChain中设计用于辅助这种转换的概念。它们接收原始用户输入并返回准备传递给语言模型的数据（即提示）。

让我们在此创建一个提示模板。该模板将接收两个用户变量：

- `language`: 要将文本翻译成的目标语言
- `text`: 要翻译的文本

In [5]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

请注意，`ChatPromptTemplate` 支持在单个模板中使用多种[消息角色](/docs/concepts/messages/#role)。我们将 `language` 参数格式化为系统消息，而用户输入的 `text` 则格式化为用户消息。

此提示模板的输入是一个字典。我们可以单独使用这个提示模板，看看它本身能实现什么功能

In [6]:
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

我们可以看到它返回了一个由两条消息组成的 `ChatPromptValue`。如果我们想直接访问这些消息，可以这样做：

In [7]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

最后，我们可以在格式化后的提示上调用聊天模型：

In [8]:
response = model.invoke(prompt)
print(response.content)

Ciao!


:::提示
消息 `content` 可以包含文本和具有附加结构的[内容块](/docs/concepts/messages/#aimessage)。更多信息请参阅[本指南](/docs/how_to/output_parser_string/)。
:::

如果我们查看 [LangSmith 追踪记录](https://smith.langchain.com/public/3ccc2d5e-2869-467b-95d6-33a577df99a2/r)，就能清晰地看到聊天模型接收的具体提示内容，以及 [token 使用情况](/docs/concepts/tokens/)、延迟时间、[标准模型参数](/docs/concepts/chat_models/#standard-parameters)（如 temperature）等其他信息。

## 结论

就是这样！在本教程中，你已经学会了如何创建第一个简单的LLM应用程序。你掌握了如何运用语言模型、如何创建提示模板，以及如何通过LangSmith为你构建的应用程序实现出色的可观测性。

这仅仅触及了成为熟练AI工程师所需学习的皮毛。幸运的是——我们还有很多其他资源！

要进一步了解LangChain的核心概念，我们提供了详细的[概念指南](/docs/concepts)。

如果您对这些概念有更具体的问题，请查阅操作指南的以下部分：

- [聊天模型](/docs/how_to/#chat-models)
- [提示词模板](/docs/how_to/#prompt-templates)

以及LangSmith文档：

- [LangSmith](https://docs.smith.langchain.com)